In [1]:
import os
import pandas as pd
from dotenv import load_dotenv
from sqlalchemy import create_engine, text, MetaData, Table, Column, String
from sqlalchemy.exc import SQLAlchemyError

load_dotenv()

# Connection configuration
user = 'root'
password = os.getenv('mysql_pass')
host = 'localhost'
database = 'Dhardware'

# Create the database connection (without specifying the database)
engine = create_engine(f'mysql+mysqlconnector://{user}:{password}@{host}')

# Create the database if it doesn't exist
with engine.connect() as conn:
    conn.execute(text(f"CREATE DATABASE IF NOT EXISTS {database}"))

# Now connect to the specific database
engine = create_engine(f'mysql+mysqlconnector://{user}:{password}@{host}/{database}')

# Directory containing the CSV files
directory = 'dataset'

# Batch size for insertion
CHUNKSIZE = 10000

# Iterate over all files in the directory
for file in os.listdir(directory):
    if file.endswith('.csv'):
        # Build the full path of the file
        file_path = os.path.join(directory, file)
        
        try:
            # Get the table name (using the file name without the extension)
            table_name = os.path.splitext(file)[0]
            
            # Read a small chunk to get the column names
            chunk = next(pd.read_csv(file_path, chunksize=1))
            column_names = chunk.columns.tolist()
            
            # Create the table if it doesn't exist
            metadata = MetaData()
            table = Table(table_name, metadata,
                          *(Column(name, String(255)) for name in column_names))
            metadata.create_all(engine)
            
            print(f"Table {table_name} created (if it didn't exist).")
            
            # Read the CSV file in batches
            for chunk in pd.read_csv(file_path, chunksize=CHUNKSIZE):
                print(f"Importing batch from {file} to table {table_name}...")
                
                # Create a new connection for each transaction
                with engine.connect() as conn:
                    # Start a transaction
                    with conn.begin():
                        # Import the DataFrame to MySQL
                        chunk.to_sql(table_name, con=conn, if_exists='append', index=False)
            
            print(f"Import of {file} completed.")
        except SQLAlchemyError as e:
            print(f"SQLAlchemy error while processing the file {file}: {e}")
        except Exception as e:
            print(f"Unexpected error while processing the file {file}: {e}")

print("Process complete. All CSV files have been imported.")

Table categories created (if it didn't exist).
Importing batch from categories.csv to table categories...
Import of categories.csv completed.
Table customers created (if it didn't exist).
Importing batch from customers.csv to table customers...
Import of customers.csv completed.
Table orders created (if it didn't exist).
Importing batch from orders.csv to table orders...
Import of orders.csv completed.
Table products created (if it didn't exist).
Importing batch from products.csv to table products...
Import of products.csv completed.
Process complete. All CSV files have been imported.
